# Business Understanding

Our company is strategically venturing into the realm of original video content creation by establishing a movie studio, recognizing the trend set by large corporations. To ensure the success of this endeavor, I am undertaking a rigorous data analysis to identify the most appealing genres for our target audience, drawing insights from the preferences of viewers. The objective is to provide actionable recommendations to the studio head, enabling informed decisions regarding the types of films to produce for optimal audience engagement and commercial success. Through this data-driven approach, I aim to position our studio as a competitive player in the dynamic landscape of digital entertainment.

# Data Understanding

## 1. Data Preparation
Loading data files to Notebook.

In [1]:
#Importing necessary libraries for the beginning of the data analysis
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt


import seaborn as sns

Loading a SQL database file by IMDB:

In [2]:
#Creating a connection with the SQL data file
connection = sqlite3.connect('im.db')
sql_query = """SELECT name FROM sqlite_master  
  WHERE type='table';"""
cursor = connection.cursor()
pd.read_sql(sql_query, connection) #reading the sql data file with pandas function
#the names of the different tables in the dataframe will be shown below

,name
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


Loading the CSV files:

In [3]:
#Using panda's read_csv to open different data files needed for the analysis
box_office= pd.read_csv('bom.movie_gross.csv.gz')
thenumbers=pd.read_csv('tn.movie_budgets.csv.gz')
moviedb=pd.read_csv('tmdb.movies.csv.gz')
rt_info= pd.read_csv('rt.movie_info.tsv.gz', compression='gzip', delimiter='\t')
rt_reviews=pd.read_csv('rt.reviews.tsv.gz', compression='gzip', delimiter='\t', encoding='latin1')

FileNotFoundError: [Errno 2] No such file or directory: 'bom.movie_gross.csv.gz'

Now that the data is loaded into this notebook, we can proceed with the data cleaning process for each data file. 

## 2. Data Cleaning
Cleaning the data files that will be used for this analysis.

I will start the Data Cleaning process separating the 5 data files in 3 sections. To get more detailed information you can click on the dropdown button or click on the title to open the detailed lines of code used. Each line of code has specific detailed on what process is happening per line.

### SQL IMDB Data File
Starting with the exploration of the SQL datafile.\
Selecting the ratings on movies in the US, filtering the number of votes to over 300 votes to be considered and focusing on movies relevant in the past 24 years.\
A new panda dataframe is created with the relevant information for analysis.

In [ ]:
#Selecting relevant columns from the different tables in the Data File with the relevant filters.
#filtering number of votes to over 3000
query=""" 
SELECT r.movie_id, b.primary_title AS title , b.genres, r.averagerating AS avg_rating, 
    r.numvotes, a.region, primary_name as director, b.start_year, b.runtime_minutes AS mov_minutes 
FROM movie_ratings r 
JOIN movie_basics b ON r.movie_id=b.movie_id 
JOIN movie_akas a ON a.movie_id=b.movie_id 
JOIN directors d ON d.movie_id=a.movie_id 
JOIN persons p ON p.person_id=d.person_id 
WHERE r.numvotes > 3000 AND region='US' AND start_year >= 2000  
ORDER BY averagerating DESC
;"""
#Creating a panda dataframe 'df' of the SQL data file
df=pd.read_sql(query,connection)

In [ ]:
#Dataframe starts with 20,832 rows of data
df.info()

In [ ]:
#Dropping duplicates from the movie_id column
df=df.drop_duplicates(subset=['movie_id'])

In [ ]:
#dropping the duplicates of 'movie_id' we get 3,391 unique rows of data
df.info()

In [ ]:
#Changing the value type of 'mov_minutes' and 'numvotes' to integer
df['mov_minutes'] = df['mov_minutes'].astype(int)
df['numvotes'] = df['numvotes'].astype(int)

In [ ]:
#Dropping duplicates from the column title, 19 rows dropped (0.006% of the rows)
df=df.drop_duplicates(subset=['title'])

In [ ]:
#Checking for NAs in the dataframe
df.isna().sum()

In [ ]:
#checking the information per column
df.info()

A recommendation will be made based on the genre of movies using this dataframe, therefore I will be filtering the most relevant genres that are needed for the analysis.

In [ ]:
#The count of values in the genres column goes from 1 to 161
df['genres'].value_counts()

Given that the value count of genres is from 1 to 161, I will be using the most relevant genres. I estimate that analyzing genres with count over 50 will be the best approach.

In [ ]:
#creating the variable genre_counts with the value_counts function
genre_counts = df['genres'].value_counts()
#creating a list of genres that have a count greater than 50
genres_over50 = genre_counts[genre_counts > 50].index.tolist()
#using the isin() method to filter the dataframe based on the selected genres over 50 count
df1 = df[df['genres'].isin(genres_over50)]    

In [ ]:
#Checking the cleaned dataframe from IMDB 
#new name of the filtered dataframe
df1.head()

### Rotten Tomatoes Data Files

Getting detailed information of the Rotten Tomatoes Info data file per column.
The rt_info data frame has 1560 rows and 12 columns of data.

In [ ]:
#checking info per column
rt_info.info()

In [ ]:
#Changing the date column to datetime format
rt_info['theater_date'] = pd.to_datetime(rt_info['theater_date'])

In [ ]:
#creating new columns for month and year
rt_info['theater_month']=rt_info['theater_date'].dt.month
rt_info['theater_year']=rt_info['theater_date'].dt.year

In [ ]:
#Filtering movies from 2000 until now in the 'theater_year' column
rt_info=rt_info[(rt_info['theater_year']>=2000)]

In [ ]:
#dropping NA values from the box office column
rt_info=rt_info.dropna(subset=['box_office'])

To not lose valuable information I will fill the corresponding director, writer and studio columns NA values with the string 'Not Available'.

In [ ]:
#replacing NA values with string
rt_info.loc[:, 'director'] = rt_info['director'].fillna('Not Available')
rt_info.loc[:, 'studio'] = rt_info['studio'].fillna('Not Available')

In [ ]:
#dropping NA values from the runtime column
rt_info.dropna(subset=['runtime'], inplace=True)

In [ ]:
#Dropping columns that are not necessary for analysis
rt_info.drop(columns=['theater_date', 'dvd_date', 'writer', 'synopsis', 'currency'], inplace=True)

In [ ]:
#checking for the NA values per column
rt_info.isna().sum()

In [ ]:
#checking the value count in the runtime column
rt_info['runtime'].value_counts()

In [ ]:
# Extracting the first numbers from the runtime column
rt_info['runtime']=rt_info['runtime'].str.extract('(\d+)').astype(int)

In [ ]:
#changing the box office, month and year columns to integer type value for easier analysis
rt_info['box_office'] = rt_info['box_office'].str.replace(',', '').astype(int)
rt_info['theater_month']=rt_info['theater_month'].astype(int)
rt_info['theater_year']=rt_info['theater_year'].astype(int)

In [ ]:
#setting the float format from the dataframe to display numbers with 2 decimal places
pd.options.display.float_format = '{:.2f}'.format

#calling the describe info
rt_info.describe()

In the rating column, I will be filtering out the ratings that are not part of the Film Rating System.

In [ ]:
#checking the values and counts from the rating column
rt_info['rating'].value_counts()

#keeping values that are not NR and NC17 because they are not considered ratings in the film rating system
rt_info = rt_info[~rt_info['rating'].isin(['NR', 'NC17'])]

#checking the values
print(rt_info['rating'])

2nd Data file from rotten tomatoes website.

Starting the data cleaning of Rotten Tomatoes Reviews. The data file contains 54,432 rows of data, and 8 columns of information.

In [ ]:
#Checking for missing information per column
rt_reviews.info()

I don't want to lose the rating of a movie just because the critic did not leave a review. So the missing reviews will be filled with the word 'None'.

In [ ]:
#Changing missing reviews to 'None'
rt_reviews['review']=rt_reviews['review'].fillna('None')

In [ ]:
#dropping the NAs in the publisher column
rt_reviews.dropna(subset=['publisher'], inplace=True)

NA values in the rating column that are missing will be dropped. There is not a way to validate/confirm the 'fresh' column without the rating.

In [ ]:
#Dropping NAs in rating column
rt_reviews.dropna(subset=['rating'], inplace=True)

In [ ]:
#Dropping NAs from the critics column
rt_reviews.dropna(subset=['critic'], inplace=True)

In [ ]:
#checking for the NAs in all the columns
rt_reviews.isna().sum()

Changing the date column values to data time and separating it by month and year for better analysis. Then I will filter out, movies from the 2000 to now.

In [ ]:
#using date time function to change the date column
rt_reviews['date']=pd.to_datetime(rt_reviews['date'])

In [ ]:
#separating month and year columns from the date column
rt_reviews['month']=rt_reviews['date'].dt.month
rt_reviews['year']=rt_reviews['date'].dt.year

In [ ]:
#filtering data frame to rows with information from the 2000's to now.
rt_reviews=rt_reviews[(rt_reviews['year']>=2000)]

In [ ]:
#Dropping reviews with no rating, they dont add value to the data file
rt_reviews['rating']=rt_reviews.dropna(subset=['rating'], inplace=True )

In [ ]:
#checking the head of the dataframe 'rt_reviews'
rt_reviews.head(5)

### Business/Revenue Related Data Files


I will start the data cleaning process by checking the information on the Data File of "The Numbers" a website which tracks business information on movies.

In [ ]:
#Dataframe contains 5,782 rows of information and 6 columns.
thenumbers.info()

In [ ]:
#Dropping duplicates using the column 'movie'
thenumbers=thenumbers.drop_duplicates(subset=['movie'])

In [ ]:
#Dropping movies that have a domestic and worlwide gross revenue of $0
thenumbers=thenumbers[thenumbers['domestic_gross'] != '$0']
thenumbers=thenumbers[thenumbers['worldwide_gross'] != '$0']

In [ ]:
#changing the 'release_date' column to date time format
thenumbers['release_date'] = pd.to_datetime(thenumbers['release_date'])

In [ ]:
#Creating new columns for year and month of the release of a movie
thenumbers['release_month']=thenumbers['release_date'].dt.month
thenumbers['release_year']=thenumbers['release_date'].dt.year

In [ ]:
#Filtering the data frame to include only movies from the 2000's to now
thenumbers=thenumbers[(thenumbers['release_year'] >= 2000)]

In [ ]:
#changing the columns from float to integer
thenumbers['production_budget']=thenumbers['production_budget'].replace('[\$,]', '', regex=True).astype(int)
thenumbers['domestic_gross']=thenumbers['domestic_gross'].replace('[\$,]', '', regex=True).astype(int)
thenumbers['worldwide_gross']=thenumbers['worldwide_gross'].replace('[\$,]', '', regex=True).astype(int)

In [ ]:
#Setting the display format for floating-point numbers to show 2 decimal places.
pd.set_option('display.float_format', '{:.2f}'.format)

In [ ]:
#checking for a clean dataframe
thenumbers.isna().sum()

In [ ]:
#checking the dataframe's head rows
thenumbers.head(5)

Starting the data cleansing process for the data file from "The Box Office Mojo" Website which tracks box-office revenue.

In [ ]:
#getting information from the dataframe
box_office.head()

In [ ]:
#Dropping duplicates from the dataframe
box_office=box_office.drop_duplicates(subset=['title'])

In [ ]:
#checking sum of NA values per column
box_office.isna().sum()

In [ ]:
box_office.info()

In [ ]:
#Dropping NA values from the dataframe if they are NA in the column
box_office=box_office.dropna(subset=['studio'])
box_office=box_office.dropna(subset=['domestic_gross'])

In [ ]:
#Changing the value type to integer for better/easier analysis
box_office['domestic_gross']=box_office['domestic_gross'].astype(int)

In [ ]:
#Fill the NA values of foreign gross with 0 to not lose a great part of data
box_office['foreign_gross']=box_office['foreign_gross'].fillna('0')

In [ ]:
#Changing the foreign gross value type to integer
box_office['foreign_gross'] = box_office['foreign_gross'].replace(',', '', regex=True)
box_office['foreign_gross'] = box_office['foreign_gross'].str.split('.').str[0]  # Remove decimal places
box_office['foreign_gross'] = box_office['foreign_gross'].astype(int)  # Convert to integers

In [ ]:
#checking one last time the sum of NA values per column
box_office.isna().sum()

# Exploratory Analysis

### Recommendation 1: Movie Genres with the highest number of votes and rating.
Based on a IMDB database, my analysis has found that the movies with the highest votes and ratings are: Action, Adventure, Sci-Fi, Animation, Comedy, Crime and Drama.

df1 is the imdb sql dataframe
rt_info
rt_reviews
the_numbers
box_office

In [ ]:
#starting with df1 dataframe
df1.head()

Filtering the average rating of movies to be higher than 5 over 10, and the number of votes a movie has to have its over 10,000.

In [ ]:
#updating the dataframe to the filters required
df1=df1[(df1['avg_rating'] > 4 ) & (df1['numvotes'] > 10000)]

Creating a variable that will group by genres the average rating of the movies and show its statistical measures

In [ ]:
#Using groupby function with aggregation of stat measures.
rating_by_genre=df1[['genres', 'avg_rating']].groupby(['genres']).agg(['count', 'median', 'mean'])
rating_by_genre = rating_by_genre.sort_values(by=('avg_rating', 'count'), ascending=False) #Sort by rating mean in descending order
rating_by_genre

In [ ]:
df1['avg_rating'].describe()

The Mean of the Average Rating is 6.5 points (horizontal dashed red line shown on graph) and the average number of movies in a genre category that I will use will have to have a count of 39 or over to be counted as significant for my analysis; given that those are the x-values above the red line (as shown in the graph below).

In [ ]:
# Plot the line chart
fig, ax = plt.subplots(figsize=(17, 7))
rating_by_genre['avg_rating']['mean'].plot(kind='line', marker='o', ax=ax)

# Add labels to the markers indicating the count of values
for i, genre in enumerate(rating_by_genre.index):
    count = rating_by_genre['avg_rating']['count'][i]
    ax.annotate(str(count), (i, rating_by_genre['avg_rating']['mean'][i]), xytext=(0, 10),
                textcoords='offset points', ha='center', fontsize=8)

    # Highlight markers with y value at or over 6.5
    if rating_by_genre['avg_rating']['mean'][i] >= 6.5:
        ax.plot(i, rating_by_genre['avg_rating']['mean'][i], marker='o', color='green')

# Add a horizontal line at y=6.5
ax.axhline(y=6.5, color='red', linestyle='--', label='Average Rating at y=6.5')

# Set the labels and title
ax.set_title("Movie Genre vs Rating")
ax.set_xlabel("Movie Genres")
ax.set_ylabel("Average Rating")

# Add a legend
ax.legend(['Number of Movies per Genre', 'Count of Movies per Genre', 'Average Rating at y=6.5'])

# Display the plot
plt.show()

The graph shows that the most significant Movie Genres with the highest number of Votes as well as having the highest ratings are Comedy, Drama, Romance, Action, Adventure and Sci-Fi.

I will use the average number of votes to make predictions based on it.

In [ ]:
mean_value = df1['numvotes'].mean()
print(mean_value)

Looking for movies with a mean of over 95,000 number of votes.\
I will be grouping the genres by the count, median and mean of the number of votes those movies got to get a viualization from this statistical summary.

In [ ]:
#grouping the genres and numvotes columns as votes by genre
votes_by_genre=df1[['genres', 'numvotes']].groupby(['genres']).agg(['count', 'median', 'mean'])
votes_by_genre = votes_by_genre.sort_values(by=('numvotes', 'mean'), ascending=False) #descencing order needed

In [ ]:
# Create the line plot
votes_by_genre_plot = votes_by_genre['numvotes']['mean'].plot(kind='line', marker='o', figsize=(18, 6))

# Set the title, xlabel, and ylabel
votes_by_genre_plot.set_title("Most watched Movie Genres based on IMDB Number of Votes")
votes_by_genre_plot.set_xlabel("Movie Genres")
votes_by_genre_plot.set_ylabel("Number of Votes Count")

# Add the horizontal line
votes_by_genre_plot.axhline(y=95000, color='red', linestyle='--')

# Highlight the marker if y value is over 130,000 and label the markers
for x, y in zip(votes_by_genre_plot.get_lines()[0].get_xdata(), votes_by_genre_plot.get_lines()[0].get_ydata()):
    if y > 95000:
        votes_by_genre_plot.plot(x, y, marker='o', markersize=10, color='red')
        votes_by_genre_plot.annotate(str(round(y)), (x, y), xytext=(0, 10), textcoords='offset points', ha='center')

plt.show()

The most watched movies bases on the number of votes these movies got are: Action, Adventure, Sci-Fi, Animation, Comedy, Crime and Drama.\
This result correlates with the first finding that analyzed the rating of movies based on movie genre.

### Recommendation 2:  Best Months of the Year to release Movies
Based on box-office tickets information from the Rotten Tomatoes database, my analysis has found the best months to release movies during hot or cold seasons.

In [ ]:
#getting a look into the dataframe
rt_info.head()

In [ ]:
#starting with 310 rows of data
rt_info.info()

Checking the measures of central tendencies of the "box_office" column to make predictions on how to move forward with the analysis of these data files.


In [ ]:
#getting the average return on box_office values
rt_info['box_office'].describe()

When dealing with a dataset that has a large standard deviation, the choice of mean can be influenced by the presence of outliers or skewed data. As seen on the box plot below. \
Box Plot of the distribution of values in the column "box office". Highlighted in read are the outliers.

In [ ]:
# Box plot with highlighted outliers
plt.boxplot(rt_info['box_office'], flierprops=dict(marker='o', markerfacecolor='red', markersize=8))
plt.ylabel('Box Office')
plt.title('Box Plot of Box Office')
plt.show()

Checking for the distribution of the data, asummning from the box plot that is not normally distributed.\

From the graphs below, the distribution is skewed to the right. This means that the majority of the data is concentrated towards the lower values, with a long tail extending towards the higher values.
This type of distribution is a right-skewed distribution. 

I will confirm if the distribution is indeed skewed using the skew() function in pandas.

In [ ]:
#creating a figure with two subplots
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

#Plotting the KDE plot on the first subplot
sns.kdeplot(rt_info['box_office'], ax=axes[0])
axes[0].set_xlabel('Box Office')
axes[0].set_ylabel('Density')
axes[0].set_title('Kernel Density Estimation Plot of Box Office')

#Plotting the histogram on the second subplot
axes[1].hist(rt_info['box_office'], bins=10)
axes[1].set_xlabel('Box Office')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Histogram of Box Office')

#adjusting the spacing between subplots
plt.tight_layout()

#show the figure
plt.show()

In [ ]:
#checking for skewness with skew function
skewness = rt_info['box_office'].skew()

if skewness > 0:
    print("The distribution is positively skewed.")
elif skewness < 0:
    print("The distribution is negatively skewed.")
else:
    print("The distribution is approximately symmetric.")

In this case is recommended to use the median as a measure of central tendency instead of the mean.
Given that the median (50th percentile) for the "box_office" column is 15,671,532.50, it may be a more appropriate measure of central tendency to use in this case.

In [ ]:
#filtering the dataframe to rows where the bpx_office return/value is over 15,671,533 (median)
filtered_rt_info=rt_info[rt_info['box_office']>15671533]
filtered_rt_info.head()

In [ ]:
#Using groupby function with aggregation of stat measures.
grouped_rtinfo=filtered_rt_info[['theater_month', 'box_office']].groupby(['theater_month']).agg(['count', 'median', 'mean'])
grouped_rtinfo = grouped_rtinfo.sort_values(by=('box_office', 'mean'), ascending=False) #descencing order needed
grouped_rtinfo

In [ ]:
#getting the mean of box office column
filtered_rt_info['box_office'].mean()

In [ ]:
# Create a copy of the DataFrame
filtered_rt_info = filtered_rt_info.copy()

# Convert theater_month column to datetime format
filtered_rt_info.loc[:, 'theater_month'] = pd.to_datetime(filtered_rt_info['theater_month'], format='%m')

# Group by month and calculate the average box office
#1.  box_office_by_month = filtered_rt_info.groupby(filtered_rt_info['theater_month'].dt.strftime('%B'))['box_office'].mean()

# Plot the bar graph
#2.  box_office_by_month.plot(kind='bar')
# Reindex the series to have months in the correct order
box_office_by_month = box_office_by_month.reindex(['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'])

# Plot the bar graph
box_office_by_month.plot(kind='bar')

plt.xlabel('Month')
plt.ylabel('Average Box Office')
plt.title('Average Box Office by Month')

# Format y-axis labels as millions
plt.ticklabel_format(axis='y', style='plain', useOffset=False, scilimits=(6, 6))

# Add a horizontal line at y=75778279
plt.axhline(y=75778279, color='red', linestyle='--', label='Threshold')

plt.legend()

According to the data analysis of the data file, the best months to release films are May through July for the warm season and November-December during the cold season.

### Recommendation 3: Movies as a Financial Success
The success of a movie release is by showing if the Gross Revenue is equal to the double of the movie's budget.\
Using the 'thenumbers' data file, merged with the IMDB and '' I will be making a recommendation.

I will be using 'the numbers' data file to analyze the Movies with highest Profit.

In [ ]:
#Getting the information of this dataframe
thenumbers.info() #3,839 rows

In [ ]:
#Create a new column called 'total_gross' by adding the 'domestic_gross' and 'world_gross' columns
#thenumbers['total_gross'] = thenumbers['domestic_gross'] + thenumbers['worldwide_gross']

In [ ]:
# Filter the DataFrame based on the condition of only financially successful movies
filtered_thenumbers = thenumbers[thenumbers['worldwide_gross'] >= 2 * thenumbers['production_budget']]
filtered_thenumbers #1,843 rows of data

In [ ]:
#checking for stats measurements of the worlwide gross sale column
filtered_thenumbers['worldwide_gross'].describe()

When dealing with a dataset that has a large standard deviation, the choice of mean can be influenced by the presence of outliers or skewed data. As seen on the box plot below. \
Box Plot of the distribution of values in the column "worldwide_gross". Highlighted in read are the outliers.

In [ ]:
# Box plot with highlighted outliers
plt.boxplot(filtered_thenumbers['worldwide_gross'], flierprops=dict(marker='o', markerfacecolor='red', markersize=8))
plt.ylabel('Box Office')
plt.title('Box Plot of Box Office')
plt.show()

Checking for the distribution of the data, asummning from the box plot that is not normally distributed. 

From the graphs below, the distribution is skewed to the right. This means that the majority of the data is concentrated towards the lower values, with a long tail extending towards the higher values.
This type of distribution is a right-skewed distribution. 

I will confirm if the distribution is indeed skewed using the skew() function in pandas.

In [ ]:
# Generate a histogram
# Generate a histogram
filtered_thenumbers['worldwide_gross'].plot(kind='hist', bins=20)
plt.title('Distribution of Total Gross')
plt.xlabel('Total Gross')
plt.ylabel('Frequency')
plt.show()

In [ ]:
#checking for skewness with skew function
skewness = filtered_thenumbers['worldwide_gross'].skew()

if skewness > 0:
    print("The distribution is positively skewed.")
elif skewness < 0:
    print("The distribution is negatively skewed.")
else:
    print("The distribution is approximately symmetric.")

In [ ]:
#using describe function to get the median of worldwide gross
filtered_thenumbers['worldwide_gross'].describe()

In [ ]:
filtered_thenumbers.head(3)

Right-skewed data shown in the graph. The median will be used to evaluate the Most Popular movies. Median is 102,404,019.\
Only the most profitable movies will be analyzed. 

In [ ]:
#filtered_thenumbers[filtered_thenumbers['worldwide_gross']>=102404019]['release_month'].value_counts() #1,175 rows

In [ ]:
#filtered_thenumbers[filtered_thenumbers['movie']=='']

In [ ]:
merged_df = pd.merge(filtered_thenumbers, df1, left_on='movie', right_on='title')
merged_df.head(3)

In [ ]:
merged_df.info() # 306 rows on 1st merge

In [ ]:
#selected_df = merged_df.loc[:, ['movie', 'release_month', 'release_year', 'total_gross', 'genres', 'avg_rating', 'numvotes', 'mov_minutes']]
merged_df= merged_df.loc[:, ['movie', 'release_month', 'release_year', 'worldwide_gross', 'genres', 'avg_rating', 'numvotes', 'mov_minutes']]

In [ ]:
merged_df.head(3)

In [ ]:
#Will merge the box_office datafile to increase the data rows for better analysis
box_office.head()

In [ ]:
#Dropping unnecesary info from the movie titles so merge can go on smoothly.
box_office['title'] = box_office['title'].str.replace(r'\(.*\)', '')

In [ ]:
#creating a worldwide_gross column that adds the domestic and foreign gross profits of a movie.
box_office['worlwide_gross']=box_office['domestic_gross'] + box_office['foreign_gross']


In [ ]:
#changing title to movie
box_office = box_office.rename(columns={'title': 'movie'})

In [ ]:
box_office.info()

In [ ]:
#MERGE OR DONT MERGE?
merged_df = pd.merge(merged_df, box_office, on='movie', how='outer')

In [ ]:
merged_df.head(2)

In [ ]:
merged_df.info()

In [ ]:
merged_df=merged_df.drop_duplicates(subset=['movie'])

In [ ]:
merged_df.info()

In [ ]:
plt.figure(figsize=(12, 6))
sns.barplot(x='genres', y='total_gross', data=selected_df)
plt.xticks(rotation=90)
plt.show()


#Scatter Plot:
#python
plt.figure(figsize=(10, 6))
sns.scatterplot(x='mov_minutes', y='avg_rating', data=selected_df)
plt.show()


# Conclusions

# Limitations

# Recommendations

# Next Steps